In [29]:
from ultralytics import YOLO
import cv2
import os 
os.chdir(r'C:\Users\Ziad\Desktop\Real projects folder\licences plate detector using YOlO v8')
from sort.sort import *
from util import get_car,read_license_plate,write_csv

results={} #save all the information we want to save in csv file

mot_tracker=Sort()

#load model
coco_model=YOLO('yolov8n.pt')#to detect the car 
licence_plate_detector=YOLO('best.pt')#the model i have trained to detect the licence


#load video
cap=cv2.VideoCapture('test2.mp4')

#list of the classes in coco data set that we want to be detected

vechales=[2,3,5,7]  #car,motorbike,bus,truck

#read frames
ret=True
frame_num=-1
while ret:
    frame_num+=1
    ret,frame=cap.read()
    results[frame_num]={}
    vechales_=[]#to save only the classes that we want
    if ret:
        detections=coco_model(frame)[0]
        for detection in detections.boxes.data.tolist():
            #print(detection)
             
            x1,y1,x2,y2,score,class_id=detection
            if int(class_id) in vechales:
                vechales_.append([x1,y1,x2,y2,score])
        
        #traking the cars make spacific id for each car in the video
        track_id=mot_tracker.update(np.asarray(vechales_))
        
        
        #detect the licence plate from the pretrained model
        licenes=licence_plate_detector(frame)[0]
        for lincen in licenes.boxes.data.tolist():
            x1,y1,x2,y2,score,class_id=lincen
            
            #make function to spacefic each licence plate with it's car
            xcar1,ycar1,xcar2,ycar2,car_id =get_car(lincen,track_id) #will return the cordinate of the car that the license plate belong to
            
            
            #crop license plate
            license_plate_crop=frame[int(y1):int(y2),int(x1):int(x2)]
            
            
            #process license plate 
            license_plate_crop_gray=cv2.cvtColor(license_plate_crop,cv2.COLOR_BGR2GRAY)
            _,license_plate_crop_threshed=cv2.threshold(license_plate_crop_gray,64,255,cv2.THRESH_BINARY_INV)
            
            
            #read license plate number
            license_plate_text,license_plate_text_score=read_license_plate(license_plate_crop_threshed)
            
            if license_plate_text is not None:
                results[frame_num][car_id] = { 'car' :{'bbox':[xcar1,ycar1,xcar2,ycar2]},
                                                     'license_plate': {'bbox':[x1,y1,x2,y2],
                                                                       'text':license_plate_text,
                                                                       'bbox_score':score,
                                                                       'text_score':license_plate_text_score}}
                
                
                
                

#write csv
write_csv(results,'./test.csv')


0: 384x640 21 cars, 1 bus, 2 trucks, 240.6ms
Speed: 20.7ms preprocess, 240.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 393.3ms
Speed: 0.0ms preprocess, 393.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 169.3ms
Speed: 0.0ms preprocess, 169.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 354.1ms
Speed: 15.6ms preprocess, 354.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 115.9ms
Speed: 0.0ms preprocess, 115.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 280.7ms
Speed: 0.0ms preprocess, 280.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 cars, 1 bus, 1 truck, 140.1ms
Speed: 0.0ms preprocess, 140.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 L